In [ ]:
# from finn.util.visualization import showInNetron
# showInNetron("output_mobilenetw5a5_ZCU102/intermediate_models/step_create_dataflow_partition.onnx")

In [ ]:
# showInNetron("../cnv_small_build/output_driver/intermediate_models/step_convert_to_hls.onnx")

In [ ]:
# showInNetron("end2end_mobilenet_tidy.onnx")

In [64]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
from finn.util.basic import alveo_default_platform
import os
import shutil

In [65]:
from custom_steps import (
    step_mobilenet_streamline,
    step_mobilenet_convert_to_hls_layers,
    step_mobilenet_convert_to_hls_layers_separate_th,
    step_mobilenet_lower_convs,
    step_mobilenet_slr_floorplan,
)

In [66]:
def select_build_steps():
    return [
        step_mobilenet_streamline,
        step_mobilenet_lower_convs,
        step_mobilenet_convert_to_hls_layers_separate_th,
        "step_create_dataflow_partition",
        "step_apply_folding_config",
        "step_generate_estimate_reports",
        "step_hls_codegen",
        "step_hls_ipgen",
        "step_set_fifo_depths",
        "step_create_stitched_ip",
        "step_synthesize_bitfile",
        "step_make_pynq_driver",
        "step_deployment_package",
    ]

In [79]:
# build driver
model_name="end2end_good_mobilenet2_wopepro_top1_tidy"
platform_name="ZCU102"
model_file = "end2end_good_mobilenet2_wopepro_top1_tidy.onnx"

# ZCU102_mobilenet_ready_folding_config

cfg = build_cfg.DataflowBuildConfig(
        steps=select_build_steps(),
        output_dir="output_%s_%s" % (model_name, platform_name),
        folding_config_file="ZCU102_mobilenet_ready_folding_config.json",
        synth_clk_period_ns=5.4,
        board = "ZCU102",
        # folding config comes with FIFO depths already
        auto_fifo_depths=False,
        shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
        generate_outputs=[
            build_cfg.DataflowOutputType.PYNQ_DRIVER,
            build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
            build_cfg.DataflowOutputType.BITFILE,
            build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
        ],
    )

In [80]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from end2end_good_mobilenet2_wopepro_top1_tidy.onnx
Intermediate outputs will be generated in /media/nvme/tmp
Final outputs will be generated in output_end2end_good_mobilenet2_wopepro_top1_tidy_ZCU102
Build log is at output_end2end_good_mobilenet2_wopepro_top1_tidy_ZCU102/build_dataflow.log
Running step: step_mobilenet_streamline [1/13]
Running step: step_mobilenet_lower_convs [2/13]
Running step: step_mobilenet_convert_to_hls_layers_separate_th [3/13]
Running step: step_create_dataflow_partition [4/13]
Running step: step_apply_folding_config [5/13]
Running step: step_generate_estimate_reports [6/13]
Running step: step_hls_codegen [7/13]
Running step: step_hls_ipgen [8/13]
Running step: step_set_fifo_depths [9/13]
Running step: step_create_stitched_ip [10/13]
Running step: step_synthesize_bitfile [11/13]
Running step: step_make_pynq_driver [12/13]
Running step: step_deployment_package [13/13]
Completed successfully
CPU times: user 3min 3s, sys: 2.16 s, tot

0

In [ ]:
# make archieve of the driver

In [81]:
from shutil import make_archive
make_archive('deploy-good-mobilenet-woprepro-top1-2bit-on-pynq', 'zip', "output_end2end_good_mobilenet2_wopepro_top1_tidy_ZCU102"+"/deploy")

'/workspace/finn/notebooks/mobilenet/deploy-good-mobilenet-woprepro-top1-2bit-on-pynq.zip'